In [ ]:
#create table between two transcriptomes for capsaspora owzarzaki

In [40]:
#python: 3.9.1
#not 3.9.6, 3.11.0., 3.10.0.,
import scanpy as sc
from Bio import SeqIO
import os
import pandas as pd
adata = sc.read_10x_h5("/Users/work/Library/Mobile Documents/com~apple~CloudDocs/Desktop/ADesktop/Studium/PhD/unicellulars/capsaspora_gex/filtered_feature_bc_matrix.h5")
genes = list(adata.var.index)

capsa_new="/Users/work/Library/Mobile Documents/com~apple~CloudDocs/Desktop/ADesktop/Studium/PhD/code/reciprocal_blast_capsa/Capsaspora.transcriptome.longestIsoform.fasta"
capsa_old="/Users/work/Desktop/data/capsaspora_old/test.fasta"  # Path to your transcriptome fasta file

#proteome_file = "/Users/work/Desktop/mus_musculus/proteome/proteome_shorthand/mus_musculus.transcriptome.fasta"
#transcriptome_file = "suberites_domuncula.longest_isoform.fasta"  # Path to your transcriptome fasta file

In [29]:
output_file = "tmp/new_to_old.fasta"  # Path to the output file to save the sequences

# Open the output file for writing
with open(output_file, "w") as output_handle:
    # Iterate over the selected genes in the specified order
    for selected_gene in genes:
        # Iterate over the sequences in the transcriptome fasta file
        for record in SeqIO.parse(capsa_new, "fasta"):
            gene_name = record.id  # Assuming the gene name is the sequence identifier
            if gene_name == selected_gene:
                # Write the selected gene sequence to the output file
                SeqIO.write(record, output_handle, "fasta")
                break  # Exit the inner loop once the gene is found

#!./blast.sh tmp/myc_targets_sd.fasta tmp/1
!./blast.sh tmp/new_to_old.fasta tmp/1 && awk -F',' '!seen[$1]++' tmp/1 > tmp/2



Building a new DB, current time: 05/06/2024 19:50:28
New DB name:   /Users/work/Desktop/data/capsaspora_old/capsaspora.transcriptome.old.fasta
New DB title:  /Users/work/Desktop/data/capsaspora_old/capsaspora.transcriptome.old.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
BLAST options error: /Users/work/Desktop/data/capsaspora_old/capsaspora.transcriptome.old.fasta does not match input format type, default input type is FASTA
BLAST Database error: No alias or index file found for nucleotide database [/Users/work/Desktop/data/capsaspora_old/capsaspora.transcriptome.old.fasta] in search path [/Users/work/Library/Mobile Documents/com~apple~CloudDocs/Desktop/ADesktop/Studium/PhD/code/reciprocal_blast_capsa::]


In [41]:
file_path = "tmp/2" 

second_items = []

with open(file_path, 'r') as file:
    items=file.read().split("\n")
    for item in items[:len(items)-1]:
        second_items.append(str(item.split(",")[1]))

#os.remove(output_file)
output_file = "tmp/old_to_new.fasta"

# Create a set to store the selected gene names
selected_genes_set = set(second_items)

# Open the output file for writing
with open(output_file, "w") as output_handle:
    # Iterate over the sequences in the proteome fasta file
    for record in SeqIO.parse(capsa_old, "fasta"):
        gene_name = record.id  # Assuming the gene name is the sequence identifier
        
        # Check if the current gene is in the set of selected genes
        if gene_name in selected_genes_set:
            # Write the selected gene sequence to the output file
            SeqIO.write(record, output_handle, "fasta")
            
            # Remove the gene from the selected_genes_set
            selected_genes_set.remove(gene_name)
            
            # Check if all selected genes have been found
            if len(selected_genes_set) == 0:
                break

!./recip_blast.sh tmp/old_to_new.fasta tmp/3 && awk -F',' '!seen[$1]++' tmp/3 > tmp/4



Building a new DB, current time: 05/06/2024 20:20:19
New DB name:   /Users/work/Desktop/data/capsaspora_new/Capsaspora.transcriptome.longestIsoform.fasta
New DB title:  /Users/work/Desktop/data/capsaspora_new/Capsaspora.transcriptome.longestIsoform.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/work/Desktop/data/capsaspora_new/Capsaspora.transcriptome.longestIsoform.fasta
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 9031 sequences in 0.295429 seconds.




In [42]:
input_file1 = "tmp/2"  # Path to the first input file
input_file2 = "tmp/4"  # Path to the second input file
full_blast_file = "full_transcriptome_capsaspora.xlsx"

with open(input_file1, 'r') as file:
    list1=file.read().split("\n")
with open(input_file2, 'r') as file:
    list2=file.read().split("\n")


# Split the elements of list1 into separate columns
list1_split = [item.split(',') for item in list1]
list1_df = pd.DataFrame(list1_split, columns=['Capsa new', 'Capsa old', 'e-value'])

# Split the elements of list2 into separate columns
list2_split = [item.split(',') for item in list2]
list2_df = pd.DataFrame(list2_split, columns=['Capsa old', 'recip Capsa new', 'e-value'])

# Merge the two dataframes based on the 'Mouse ID' column
merged_df = pd.merge(list1_df, list2_df, on='Capsa old', how='left')

# Save the merged dataframe to an Excel file
merged_df.to_excel(full_blast_file, index=False)

print("Merged data saved to '%s'." % full_blast_file)
#os.remove(output_file)
#os.remove(input_file1)
#os.remove(input_file2)

#=IF(A2=D2;"success";"error")

/var/folders/tb/mtqjrwzn04v2ws67ymnhmv5h0000gp/T/ipykernel_24138/1545246581.py:23: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  merged_df.to_excel(full_blast_file, index=False)


Merged data saved to 'full_transcriptome_capsaspora.xlsx'.
